# 2-3. 간단한 위험 텍스트 감지 실습 (Jupyter Notebook)

이 노트북에서는 다음을 구현해 봅니다.

1. **위험 카테고리 정의**  
   - Violence / Hate / Self-harm / Illegal / Sexual 등

2. **pyahocorasick + regex를 이용한 위험 감지기**  
   - 여러 키워드를 한 번에 스캔하는 multi-pattern matcher  
   - 정규식을 이용해 우회 표현도 일부 포착

3. **간단한 ChatGuard 객체**  
   - 입력 텍스트를 먼저 감지기에 통과시킨 뒤  
   - 안전하면 OpenAI API를 호출하여 짧은 답변 생성 (max_tokens=128)

> 목표: **실제 서비스의 “전·후단 guard 레이어”가 어떤 역할을 하는지 체험**

## 1. 라이브러리 설치

In [1]:
!pip install -q pyahocorasick openai regex

## 2. 기본 설정 & OpenAI 클라이언트 준비

In [4]:
import os
import ahocorasick  # pyahocorasick의 모듈 이름
import regex as re
from dataclasses import dataclass
from typing import List, Dict, Any

from openai import OpenAI  

# 환경변수에서 API 키 읽기 (노트북 실행 전, 터미널에서 아래처럼 설정)
#   export OPENAI_API_KEY="sk-..."
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되어 있지 않습니다.")

client = OpenAI(api_key=api_key)

## 3. 위험 카테고리 및 키워드 / 패턴 정의

In [5]:
CATEGORIES = ["selfharm", "violence", "hate", "illegal", "sexual"]

# 간단한 예시 키워드들 (실습 중에 각자 확장해도 좋습니다)
KEYWORDS: Dict[str, List[str]] = {
    "selfharm": [
        "죽고 싶", "자해", "끝내고 싶", "살기 싫",
    ],
    "violence": [
        "죽인다", "패버려", "폭파", "폭탄", "칼로", "총으로",
    ],
    "hate": [
        "멍청한 놈", "한심한 놈", "종자", "충", ""
    ],
    "illegal": [
        "마약", "해킹", "불법 다운로드", "총기 밀매", "위조 여권",
    ],
    "sexual": [
        "야한", "19금", "성적", "노출", "음란",
    ],
}

# 우회 표현 등을 잡기 위한 간단한 regex 패턴 예시
REGEX_PATTERNS: Dict[str, List[str]] = {
    "selfharm": [
    ],
    
    "violence": [
    ],
    
    "hate": [
        r"씨[\s\*\-]*발",
        r"개[\s\-]*새끼",
        r"병[\s\-]*신",
        r"[가-힣]+\s*년",
    ],
    
    # 주민등록번호 형식 매칭 (주민번호/불법 개인정보 유출 탐지용)
    "illegal": [
        # 기본 형식: YYMMDD-XXXXXXX
        r"\b\d{6}-\d{7}\b",
        # 하이픈 없이 붙여 쓴 13자리: YYMMDDXXXXXXX
        r"\b\d{6}\d{7}\b",
        # 선택적 하이픈/공백 허용 버전 (원하면 이걸로만 사용해도 됨)
        r"\b\d{6}[-\s]?\d{7}\b",
    ],
    
    "sexual": [
    ],
}


## 4. pyahocorasick로 키워드 매칭기 만들기

In [6]:
def build_automaton(keyword_dict: Dict[str, List[str]]) -> ahocorasick.Automaton:
    """
    카테고리별 키워드 dict를 받아 Aho-Corasick automaton을 생성합니다.
    value에는 (category, keyword) 튜플을 저장합니다.
    """
    automaton = ahocorasick.Automaton()
    
    for category, words in keyword_dict.items():
        for w in words:
            automaton.add_word(w, (category, w))
    
    automaton.make_automaton()
    return automaton

A = build_automaton(KEYWORDS)

In [7]:
def find_keyword_matches(text: str, automaton: ahocorasick.Automaton):
    """주어진 text에서 automaton을 사용하여 모든 (category, keyword) 매칭 반환."""
    matches = []
    for end_idx, (category, word) in automaton.iter(text):
        start_idx = end_idx - len(word) + 1
        matches.append(
            {
                "category": category,
                "keyword": word,
                "span": (start_idx, end_idx),
            }
        )
    return matches

# 간단 테스트
test_text = "요즘 너무 힘들어서 죽고 싶다는 생각이 들고, 누굴 패버리고 싶기도 해."
find_keyword_matches(test_text, A)

[{'category': 'selfharm', 'keyword': '죽고 싶', 'span': (11, 14)}]

## 5. regex로 추가 패턴 감지하기

In [10]:
def find_regex_matches(text: str, pattern_dict: Dict[str, List[str]]):
    """카테고리별 regex pattern 리스트를 이용해 매칭 결과 반환."""
    matches = []
    for category, patterns in pattern_dict.items():
        for pattern in patterns:
            for m in re.finditer(pattern, text, flags=re.IGNORECASE):
                matches.append(
                    {
                        "category": category,
                        "pattern": pattern,
                        "span": m.span(),
                    }
                )
    return matches

# 테스트
test_text2 = "내 주민등록번호는 사실 900101-1234567 이야"
find_regex_matches(test_text2, REGEX_PATTERNS)

[{'category': 'illegal', 'pattern': '\\b\\d{6}-\\d{7}\\b', 'span': (13, 27)},
 {'category': 'illegal',
  'pattern': '\\b\\d{6}[-\\s]?\\d{7}\\b',
  'span': (13, 27)}]

## 6. 통합 위험 감지기 `detect_risk` 구현

In [13]:
@dataclass
class RiskResult:
    blocked: bool
    categories: List[str]
    keyword_matches: List[Dict[str, Any]]
    regex_matches: List[Dict[str, Any]]

def detect_risk(text: str) -> RiskResult:
    kw_matches = find_keyword_matches(text, A)
    rx_matches = find_regex_matches(text, REGEX_PATTERNS)
    
    categories = sorted(
        {m["category"] for m in kw_matches} |
        {m["category"] for m in rx_matches}
    )
    
    blocked = len(categories) > 0
    
    return RiskResult(
        blocked=blocked,
        categories=categories,
        keyword_matches=kw_matches,
        regex_matches=rx_matches,
    )

# 테스트
samples = [
    "죽고 싶다는 생각이 들 때가 있어.",
    "망할년.",
    "그냥 평범한 하루였다.",
    "신묘년을 맞아 새로운 유행어를 만들어 봤습니다. 너 뭐하묘",
]
for t in samples:
    r = detect_risk(t)
    print("문장:", t)
    print("  blocked:", r.blocked, "| categories:", r.categories)
    print("-" * 60)

문장: 죽고 싶다는 생각이 들 때가 있어.
  blocked: True | categories: ['selfharm']
------------------------------------------------------------
문장: 망할년.
  blocked: True | categories: ['hate']
------------------------------------------------------------
문장: 그냥 평범한 하루였다.
  blocked: False | categories: []
------------------------------------------------------------
문장: 신묘년을 맞아 새로운 유행어를 만들어 봤습니다. 너 뭐하묘
  blocked: True | categories: ['hate']
------------------------------------------------------------


## 7. ChatGuard 객체 구현 (검열 → OpenAI 호출)

In [14]:
@dataclass
class ChatGuard:
    model: str = "gpt-4.1-mini"  # 저렴한 모델 사용
    max_tokens: int = 128
    system_prompt: str = (
        "You are a helpful assistant. "
        "Keep your answers short and safe."
    )

    def check(self, text: str) -> RiskResult:
        return detect_risk(text)

    def chat(self, text: str) -> str:
        """
        1) 위험 감지
        2) 위험이면 차단 메시지 반환
        3) 안전하면 OpenAI API 호출
        """
        risk = self.check(text)
        if risk.blocked:
            categories_str = ", ".join(risk.categories)
            return f"⚠️ 안전 정책에 의해 차단된 요청입니다. (카테고리: {categories_str})"
        
        completion = client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": text},
            ],
            max_tokens=self.max_tokens,
            temperature=0.5,
        )
        return completion.choices[0].message.content.strip()

guard = ChatGuard()

## 8. 간단 인터랙티브 데모 함수

In [ ]:
def demo_chat():
    print("간단 ChatGuard 데모입니다. 'quit' 또는 'exit'를 입력하면 종료합니다.\n")
    while True:
        user = input("You: ")
        if user.strip().lower() in {"quit", "exit"}:
            print("종료합니다.")
            break
        reply = guard.chat(user)
        print("Assistant:", reply)
        print()

demo_chat()  

간단 ChatGuard 데모입니다. 'quit' 또는 'exit'를 입력하면 종료합니다.

Assistant: 안녕하세요! 어떻게 도와드릴까요?

Assistant: ⚠️ 안전 정책에 의해 차단된 요청입니다. (카테고리: hate)

종료합니다.


: 